<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B02%5D%20-%20Modelos%20Supervisados%20Lineales/Supervisados_Lineales_Ejercicio_4_regresion_logistica_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Supervisados Lineales - Ejercicio 4: regresion_logistica_diabetes.ipynb

En este ejercicio entrenaremos un modelo de **Regresión Logística** utilizando el dataset de Diabetes de `scikit-learn`. Convertiremos la variable objetivo a binaria para plantear un problema de clasificación, evaluaremos el rendimiento con métricas adecuadas y analizaremos la interpretabilidad de los coeficientes.

### Objetivos
- Cargar el dataset de Diabetes desde `scikit-learn`.
- Formular un problema de clasificación binaria a partir del objetivo continuo.
- Entrenar una **Regresión Logística** con y sin estandarización.
- Evaluar el modelo con Accuracy, Matriz de confusión, ROC-AUC.
- Interpretar coeficientes como odds ratios.

### Descripción del dataset
El dataset de Diabetes contiene 442 pacientes y 10 características numéricas normalizadas (edad, sexo, IMC, presión sanguínea y 6 medidas bioquímicas). El objetivo original es continuo y representa la progresión de la enfermedad tras un año. Para clasificación, lo binarizaremos.

### 1. Importación de librerías y carga del dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, classification_report
sns.set(style='whitegrid')

In [ ]:
# Cargar dataset de Diabetes
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y_cont = pd.Series(data.target, name='progression')

# Convertimos a clasificación binaria: 1 si progresión > mediana, 0 en caso contrario
threshold = y_cont.median()
y = (y_cont > threshold).astype(int)
print('Umbral de binarización (mediana):', threshold)
X.head()

### 2. División de datos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape, y_train.mean(), y_test.mean()

### 4. Estandarización y entrenamiento

In [ ]:
# Estandarizamos todas las columnas numéricas
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)


In [ ]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_std, y_train)

In [ ]:
y_pred = logreg.predict(X_test_std)
y_proba = logreg.predict_proba(X_test_std)[:, 1]

In [ ]:
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
print('Accuracy (std):', acc)
print('ROC-AUC (std):', auc)

### 5. Matriz de confusión y clasificación

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=['Real 0','Real 1'], columns=['Pred 0','Pred 1'])
print(cm_df)
print('\nReporte de clasificación:')
print(classification_report(y_test, y_pred, digits=3))

### 6. Curva ROC

In [ ]:
fpr, tpr, thr = roc_curve(y_test, y_proba)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'ROC-AUC = {auc:.3f}')
plt.plot([0,1],[0,1],'k--', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Regresión Logística (estandarizado)')
plt.legend(loc='lower right')
plt.show()

### 7. Interpretación de coeficientes

Cada variable aporta un "empuje" al resultado: positivo = aumenta probabilidad, negativo = reduce.

El odds_ratio nos dice cuánto multiplican ( >1 ) o dividen ( <1 ) las probabilidades de que ocurra el evento.


In [ ]:
# Coeficientes en escala estandarizada -> interpretables como efecto por desviación estándar
coef = pd.Series(logreg.coef_[0], index=X.columns, name='coef')
odds_ratio = np.exp(coef)
summary = pd.concat([coef, odds_ratio.rename('odds_ratio')], axis=1).sort_values('odds_ratio', ascending=False)
summary

### Conclusión
Hemos formulado un problema de clasificación a partir del objetivo continuo del dataset de Diabetes, entrenado una Regresión Logística con y sin estandarización, evaluado su rendimiento con métricas adecuadas y analizado la contribución de cada variable mediante coeficientes y odds ratios.